# Introduction to DataFrames
**[Bogumił Kamiński](http://bogumilkaminski.pl/about/), October 10, 2018**

Let's get started by loading the `DataFrames` package.

In [1]:
using DataFrames, Random

## Constructors and conversion

### Constructors

In this section, you'll see many ways to create a `DataFrame` using the `DataFrame()` constructor.

First, we could create an empty DataFrame,

In [2]:
DataFrame() # empty DataFrame

""
""


Or we could call the constructor using keyword arguments to add columns to the `DataFrame`.

In [3]:
DataFrame(A=1:3, B=rand(3), C=randstring.([3,3,3]))

,A,B,C
,Int64,Float64,String
1,1,0.976504,OsL
2,2,0.530972,3d1
3,3,0.476606,SzH


We can create a `DataFrame` from a dictionary, in which case keys from the dictionary will be sorted to create the `DataFrame` columns.

In [4]:
x = Dict("A" => [1,2], "B" => [true, false], "C" => ['a', 'b'])
DataFrame(x)

,A,B,C
,Int64,Bool,Char
1,1,true,'a'
2,2,false,'b'


Rather than explicitly creating a dictionary first, as above, we could pass `DataFrame` arguments with the syntax of dictionary key-value pairs. 

Note that in this case, we use symbols to denote the column names and arguments are not sorted. For example, `:A`, the symbol, produces `A`, the name of the first column here:

In [5]:
DataFrame(:A => [1,2], :B => [true, false], :C => ['a', 'b'])

,A,B,C
,Int64,Bool,Char
1,1,true,'a'
2,2,false,'b'


Here we create a `DataFrame` from a vector of vectors, and each vector becomes a column.

In [6]:
DataFrame([rand(3) for i in 1:3])

,x1,x2,x3
,Float64,Float64,Float64
1,0.909713,0.960152,0.118858
2,0.0524781,0.595181,0.0403506
3,0.0284224,0.138849,0.847181


It is not allow to pass a vector of scalars to `DataFrame` constructor.

In [7]:
DataFrame(rand(3))

ArgumentError: ArgumentError: unable to construct DataFrame from Array{Float64,1}

Instead use a transposed vector if you have a vector of atoms (in this way you effectively pass a two dimensional array to the constructor which is supported).

In [8]:
DataFrame(permutedims([1, 2, 3]))

,x1,x2,x3
,Int64,Int64,Int64
1,1,2,3


or pass a vector of `NamedTuple`s

In [9]:
v = [(a=1, b=2), (a=3, b=4)]
DataFrame(v)

,a,b
,Int64,Int64
1,1,2
2,3,4


Pass a second argument to give the columns names in case you pass a vector of vectors.

In [10]:
DataFrame([1:3, 4:6, 7:9], [:A, :B, :C])

,A,B,C
,Int64,Int64,Int64
1,1,4,7
2,2,5,8
3,3,6,9


Alternatively you can pass a `NamedTupe` of vectors:

In [11]:
n = (a=1:3, b=11:13)
DataFrame(n)

,a,b
,Int64,Int64
1,1,11
2,2,12
3,3,13


Here we create a `DataFrame` from a matrix,

In [12]:
DataFrame(rand(3,4))

,x1,x2,x3,x4
,Float64,Float64,Float64,Float64
1,0.967455,0.904461,0.679839,0.0816703
2,0.173464,0.731574,0.620152,0.0213068
3,0.886806,0.175889,0.0555942,0.717657


and here we do the same but also pass column names.

In [13]:
DataFrame(rand(3,4), Symbol.('a':'d'))

,a,b,c,d
,Float64,Float64,Float64,Float64
1,0.53027,0.128992,0.845584,0.375157
2,0.0527804,0.173207,0.180505,0.0299634
3,0.92046,0.225288,0.0958122,0.629859


We can also construct an uninitialized DataFrame.

Here we pass column types, names and number of rows; we get `missing` in column :C because `Any >: Missing`.

In [14]:
DataFrame([Int, Float64, Any], [:A, :B, :C], 1)

,A,B,C
,Int64,Float64,Any
1,100663296,1.25987e-321,missing


Here we create a `DataFrame` where `:C` is `#undef`

In [15]:
DataFrame([Int, Float64, String], [:A, :B, :C], 1)

,A,B,C
,Int64,Float64,String
1,75301440,4.94066e-324,#undef


To initialize a `DataFrame` with column names, but no rows use

In [16]:
DataFrame([Int, Float64, String], [:A, :B, :C], 0) 

,A,B,C
,Int64,Float64,String


or

In [17]:
DataFrame(A=Int[], B=Float64[], C=String[])

,A,B,C
,Int64,Float64,String


This syntax gives us a quick way to create homogenous `DataFrame`.

In [18]:
DataFrame(Int, 3, 5)

,x1,x2,x3,x4,x5
,Int64,Int64,Int64,Int64,Int64
1,99364752,21728602576,21731324304,1,1
2,142479296,142478912,142481600,2,2
3,142481472,142487744,142478912,3,3


This example is similar, but has nonhomogenous columns.

In [19]:
DataFrame([Int, Float64], 4)

,x1,x2
,Int64,Float64
1,21670119008,0.0
2,75302896,4.94066e-324
3,116759232,0.0
4,75301440,0.0


Finally, we can create a `DataFrame` by copying an existing `DataFrame`.

Note that `copy` creates a shallow copy.

In [20]:
x = DataFrame(a=1:2, b='a':'b')
y = copy(x)
(x === y), isequal(x, y)

(false, true)

### Conversion to a matrix

Let's start by creating a `DataFrame` with two rows and two columns.

In [21]:
x = DataFrame(x=1:2, y=["A", "B"])

,x,y
,Int64,String
1,1,A
2,2,B


We can create a matrix by passing this `DataFrame` to `Matrix`.

In [22]:
Matrix(x)

2×2 Array{Any,2}:
 1  "A"
 2  "B"

This would work even if the `DataFrame` had some `missing`s:

In [23]:
x = DataFrame(x=1:2, y=[missing,"B"])

,x,y
,Int64,String⍰
1,1,missing
2,2,B


In [24]:
Matrix(x)

2×2 Array{Any,2}:
 1  missing
 2  "B"    

In the two previous matrix examples, Julia created matrices with elements of type `Any`. We can see more clearly that the type of matrix is inferred when we pass, for example, a `DataFrame` of integers to `Matrix`, creating a 2D `Array` of `Int64`s:

In [25]:
x = DataFrame(x=1:2, y=3:4)

,x,y
,Int64,Int64
1,1,3
2,2,4


In [26]:
Matrix(x)

2×2 Array{Int64,2}:
 1  3
 2  4

In this next example, Julia correctly identifies that `Union` is needed to express the type of the resulting `Matrix` (which contains `missing`s).

In [27]:
x = DataFrame(x=1:2, y=[missing,4])

,x,y
,Int64,Int64⍰
1,1,missing
2,2,4


In [28]:
Matrix(x)

2×2 Array{Union{Missing, Int64},2}:
 1   missing
 2  4       

Note that we can't force a conversion of `missing` values to `Int`s!

In [29]:
Matrix{Int}(x)

ErrorException: cannot convert a DataFrame containing missing values to array (found for column y)

### Handling of duplicate column names

We can pass the `makeunique` keyword argument to allow passing duplicate names (they get deduplicated)

In [30]:
df = DataFrame(:a=>1, :a=>2, :a_1=>3; makeunique=true)

,a,a_2,a_1
,Int64,Int64,Int64
1,1,2,3


Otherwise, duplicates will not be allowed in the future.

In [31]:
df = DataFrame(:a=>1, :a=>2, :a_1=>3)

┌ Warning: Duplicate variable names are deprecated: pass makeunique=true to add a suffix automatically.
│   caller = ip:0x0
└ @ Core :-1


,a,a_2,a_1
,Int64,Int64,Int64
1,1,2,3


Finallly observe that `nothing` is not printed when displaying a `DataFrame`:

In [32]:
DataFrame(x=[1, nothing], y=[nothing, "a"])

,x,y
,Union…,Union…
1,1,
2,,a
